In [1]:
%reload_ext autoreload
%autoreload 2


In [2]:
import sys
sys.path.insert(0, r'/Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/build')

import numpy as np

import cpp_probabilities

## Load data

In [3]:
from result_saver import SaverProvider
provider = SaverProvider()
backend_name = provider.get_backend('ibmq_mumbai') 
job = provider.retrieve_job("cmyhbrqrmwhg008bs4h0") # Mumbai job
# job = provider.retrieve_job("cn6g47862r90008810pg") # Sherbrooke job
memory = job.result().get_memory()

print(memory.shape)


(1000, 9)


In [ ]:
from soft_info import get_KDEs, get_repcode_layout, get_repcode_IQ_map

## Mumbai
layout = [25, 19, 26, 22, 16] # To implement: into scratch job_data
kde_dict, processed_scaler_dict = get_KDEs(provider, tobecalib_job="cmyhbrqrmwhg008bs4h0")
# kde_dict, scaler_dict = get_KDEs(provider, device='ibmq_mumbai', qubits=list(range(27)), bandwidths=0.2, plot=False)
synd_rounds = 3

## Sherbrooke
# layout = get_repcode_layout(30, provider.get_backend('ibm_sherbrooke') )
# kde_dict, scaler_dict = get_KDEs(provider, 'ibm_sdherbrooke', list(range(127)), bandwidths=0.2, plot=False)
# synd_rounds = 35

print("Generating the qubit mapping...")
qubit_mapping = get_repcode_IQ_map(layout, synd_rounds) #Hardcoded for repetition codes

Generating the qubit mapping...


# How to use

In [24]:
from Scratch import create_or_load_kde_grid

# Example usage
# grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, "cmyhbrqrmwhg008bs4h0", 10, 3)
grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, "cn6h8rv62r90008814s0", 3, 3)

ImportError: cannot import name 'create_or_load_kde_grid' from 'Scratch' (/Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/src/Scratch/__init__.py)

# Metadata structure

In [17]:
import json
import pandas as pd


from datetime import datetime

import os
from typing import Dict, List, Optional, Union
from sklearn.preprocessing import StandardScaler

from Scratch import find_and_create_scratch

from cpp import process_scaler_dict

import numpy as np

def generate_kde_grid(kde_dict, num_points, num_std_dev=3):
    grid_dict = {}

    # Define the grid range and create grid points
    grid_range_real = np.linspace(-num_std_dev, num_std_dev, num_points)
    grid_range_imag = np.linspace(-num_std_dev, num_std_dev, num_points)
    grid_x, grid_y = np.meshgrid(grid_range_real, grid_range_imag)
    grid_points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T

    for qubit_idx, (kde_0, kde_1) in kde_dict.items():
        # Evaluate KDE on the grid for both states
        grid_density_0 = (kde_0.score_samples(grid_points)).reshape(grid_x.shape)
        grid_density_1 = (kde_1.score_samples(grid_points)).reshape(grid_x.shape)
        
        # Create an instance of GridData and store in dictionary
        # grid_dict[qubit_idx] = cpp_probabilities.GridData(grid_x, grid_y, grid_density_0, grid_density_1)
        grid_dict[qubit_idx] = (grid_x, grid_y, grid_density_0, grid_density_1)

    return grid_dict



def update_grid_metadata(metadata_path, creation_date, backend_name, job_ids, grid_file_path, num_grid_points, num_std_dev):
    # Convert creation_date to datetime if it is a string
    if isinstance(creation_date, str):
        creation_date = datetime.strptime(creation_date, "%y-%m-%d_%Hhh%M")

    # Check if the metadata file exists
    if os.path.exists(metadata_path):
        # Read existing metadata
        with open(metadata_path, 'r') as file:
            metadata = json.load(file)
    else:
        # Initialize metadata if file does not exist
        metadata = {}

    # Format the creation date
    formatted_date = creation_date.strftime("%y.%m.%d_%Hh%M")

    # Create a nested entry for the backend, if it doesn't exist
    if backend_name not in metadata:
        metadata[backend_name] = {}

    # Update or create the metadata entry for the specific backend and date
    formatted_key = f"{creation_date.strftime('%y.%m.%d_%Hh%M')}_{num_grid_points}pts_{num_std_dev}std"
    metadata[backend_name][formatted_key] = {
        'grid_file_path': grid_file_path,
        'job_ids': job_ids,
    }

    # Write the updated metadata back to the file
    with open(metadata_path, 'w') as file:
        json.dump(metadata, file, indent=4)



def save_grid(grid_dict, processed_scaler_dict, filename):
    """
    Save a grid dictionary and scaler dictionary as a JSON file.

    Args:
        grid_dict (dict): Dictionary containing tuples of NumPy arrays.
        scaler_dict (dict): Dictionary containing scaler parameters.
        filename (str): The name of the file to save the grid and scaler data.
    """
    serializable_grid_dict = {}
    for qubit_idx, (grid_x, grid_y, grid_density_0, grid_density_1) in grid_dict.items():
        serializable_grid_dict[qubit_idx] = {
            'grid_x': grid_x.tolist(), 
            'grid_y': grid_y.tolist(), 
            'grid_density_0': grid_density_0.tolist(), 
            'grid_density_1': grid_density_1.tolist()
        }

    data_to_save = {
        'grid_data': serializable_grid_dict,
        'scaler_data': processed_scaler_dict
    }

    with open(filename, 'w') as file:
        json.dump(data_to_save, file, indent=4)
    


def load_grid(filename):
    """
    Load a grid dictionary and scaler dictionary from a JSON file and create GridData objects.

    Args:
        filename (str): The name of the file to load the grid and scaler data from.

    Returns:
        tuple: A tuple containing two dictionaries, one with GridData objects and the other with scaler data.
    """
    with open(filename, 'r') as file:
        data_loaded = json.load(file)

    grid_dict = {}
    for qubit_idx, grid_data in data_loaded['grid_data'].items():
        grid_x = np.array(grid_data['grid_x'])
        grid_y = np.array(grid_data['grid_y'])
        grid_density_0 = np.array(grid_data['grid_density_0'])
        grid_density_1 = np.array(grid_data['grid_density_1'])

        # Create GridData objects
        grid_dict[int(qubit_idx)] = cpp_probabilities.GridData(
            grid_x, grid_y, grid_density_0, grid_density_1
        )

    scaler_dict = data_loaded['scaler_data']

    return grid_dict, scaler_dict




# # Example usage
# root_dir = find_and_create_scratch()
# grid_dict = generate_kde_grid(kde_dict, 5, num_std_dev=3)  # Replace with actual kde_dict
# save_grid(grid_dict, root_dir + '/calibration_grids/your_grid_file.json')
# grid = load_grid(root_dir + '/calibration_grids/your_grid_file.json')
# print(grid[0])



In [23]:
import os
from cpp import process_scaler_dict

from Scratch import find_closest_calib_jobs

def create_or_load_kde_grid(provider, tobecalib_job: str, num_grid_points: int, num_std_dev: int=3):
    root_dir = find_and_create_scratch()
    grid_folder = root_dir + '/calibration_grids'
    metadata_file = root_dir + '/calibration_grid_metadata.json'

    # Check if metadata file exists and load it
    if os.path.exists(metadata_file):
        with open(metadata_file, 'r') as file:
            grid_metadata = json.load(file)
    else:
        grid_metadata = {}

    # Find the closest calibration jobs
    closest_job_ids, backend, creation_time = find_closest_calib_jobs(tobecalib_job)

    # Format the creation date and construct the key for metadata
    formatted_key = f"{creation_time.strftime('%y.%m.%d_%Hh%M')}_{num_grid_points}pts_{num_std_dev}std"
    grid_entry = grid_metadata.get(backend, {}).get(formatted_key)

    

    if grid_entry:
        # Load the existing grid and scaler
        grid_file_path = grid_entry['grid_file_path']
        loaded_grid_dict, loaded_scaler_dict = load_grid(grid_file_path)
        return loaded_grid_dict, loaded_scaler_dict
    else:
        # Retrieve KDEs and scaler data
        kde_dict, scaler_dict = get_KDEs(provider, tobecalib_job=tobecalib_job)
        processed_scaler_dict = process_scaler_dict(scaler_dict)

        # Generate a new grid and save it
        grid_dict = generate_kde_grid(kde_dict, num_grid_points, num_std_dev)
        grid_file_path = os.path.join(grid_folder, f"{formatted_key}_grid.json")
        save_grid(grid_dict, processed_scaler_dict, grid_file_path)

        # Update metadata
        update_grid_metadata(metadata_file, creation_date=creation_time, backend_name=backend, job_ids=closest_job_ids, grid_file_path=grid_file_path, num_grid_points=num_grid_points, num_std_dev=num_std_dev)

    return grid_dict, processed_scaler_dict



# Example usage
# grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, "cmyhbrqrmwhg008bs4h0", 10, 3)
grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, "cn6h8rv62r90008814s0", 3, 3)


# Generate grid data

In [24]:
import numpy as np

def generate_kde_grid(kde_dict, num_points, num_std_dev=10):
    grid_dict = {}

    # Define the grid range and create grid points
    grid_range_real = np.linspace(-num_std_dev, num_std_dev, num_points)
    grid_range_imag = np.linspace(-num_std_dev, num_std_dev, num_points)
    grid_x, grid_y = np.meshgrid(grid_range_real, grid_range_imag)
    grid_points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T

    for qubit_idx, (kde_0, kde_1) in kde_dict.items():
        # Evaluate KDE on the grid for both states
        grid_density_0 = (kde_0.score_samples(grid_points)).reshape(grid_x.shape)
        grid_density_1 = (kde_1.score_samples(grid_points)).reshape(grid_x.shape)
        
        # Create an instance of GridData and store in dictionary
        grid_dict[qubit_idx] = cpp_probabilities.GridData(grid_x, grid_y, grid_density_0, grid_density_1)

    return grid_dict

grid_dict = generate_kde_grid(kde_dict, 10, num_std_dev=3)
print(len(grid_dict))


27


# Test grid data

In [25]:
from cpp import process_scaler_dict


processed_scaler_dict = process_scaler_dict(processed_scaler_dict)

counts = cpp_probabilities.get_counts(memory, qubit_mapping, grid_dict, processed_scaler_dict, synd_rounds)
(sorted(counts.items(), key=lambda x: x[1], reverse=True))

[('000 00 00 00', 866),
 ('000 01 00 00', 15),
 ('000 10 10 10', 11),
 ('000 01 01 01', 10),
 ('000 00 01 00', 9),
 ('000 10 00 00', 9),
 ('100 00 10 00', 8),
 ('100 10 00 10', 7),
 ('000 10 10 00', 6),
 ('001 00 00 00', 5),
 ('010 11 00 11', 5),
 ('000 00 10 00', 4),
 ('001 00 01 00', 4),
 ('001 01 00 00', 4),
 ('001 01 00 01', 4),
 ('100 10 00 00', 4),
 ('100 00 00 00', 3),
 ('000 00 10 10', 2),
 ('000 01 01 00', 2),
 ('000 10 00 10', 2),
 ('000 00 00 01', 1),
 ('000 00 00 10', 1),
 ('000 01 00 01', 1),
 ('000 10 11 10', 1),
 ('000 11 01 01', 1),
 ('001 01 01 00', 1),
 ('001 01 01 01', 1),
 ('001 01 10 00', 1),
 ('001 10 10 00', 1),
 ('001 11 10 10', 1),
 ('010 00 00 00', 1),
 ('010 00 11 00', 1),
 ('010 01 00 11', 1),
 ('010 01 10 01', 1),
 ('010 10 00 00', 1),
 ('010 10 01 00', 1),
 ('010 11 00 00', 1),
 ('010 11 01 00', 1),
 ('100 00 10 10', 1),
 ('100 10 10 00', 1)]